In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from delta.tables import DeltaTable

# glopal variables
storage_account = "ataacarprojectdatalake"
datalake_key = "xxxxxxxxx"
inc_flag = True
dim_name = "dim_date"
spark.conf.set(f"fs.azure.account.key.{storage_account}.dfs.core.windows.net", 
               access_key)

#extracting the model
df_src = spark.sql('''
                         select Distinct Date_ID from parquet.`abfss://silver@ataacarprojectdatalake.dfs.core.windows.net/car_sales/`
                         ''')



In [0]:
if inc_flag:
    # if we have the table - case of incremental run - then get all of the data from the gold layer
    df_target = spark.sql('''
                        select Date_sk ,Date_ID from delta.`abfss://gold@ataacarprojectdatalake.dfs.core.windows.net/dim_date/` 
                        ''')
    #get the table max value to be added layer to the next new data
    max_value = df_target.select(F.max("date_sk")).collect()[0][0]


elif not inc_flag:
    # if we don't have the table - case of full run - then get only the table schema from the silver layer
    df_target = spark.sql('''
                          select 1 as Date_sk ,Date_ID from parquet.`abfss://silver@ataacarprojectdatalake.dfs.core.windows.net/car_sales/` 
                          where 1=0
                          ''')
    max_value = 1

    

    

In [0]:
df_joined = df_src.join(df_target, df_src["Date_ID"] == df_target["Date_ID"], how='left').select(df_src["Date_ID"], df_target["Date_sk"])
df_new_source_data = df_joined.filter(df_joined["Date_sk"].isNull())
df_old_source_data = df_joined.filter(df_joined["Date_sk"].isNotNull())

df_new_source_data = df_new_source_data.withColumn("Date_sk", F.monotonically_increasing_id()+max_value)

df_model_final = df_new_source_data.union(df_old_source_data)
    

In [0]:
if inc_flag:
    delta_table = DeltaTable.forPath(spark, f"abfss://gold@ataacarprojectdatalake.dfs.core.windows.net/{dim_name}/")
    delta_table.alias("trg").merge(df_model_final.alias("src"), "trg.Date_ID = src.Date_ID")\
                .whenMatchedUpdateAll()\
                .whenNotMatchedInsertAll()\
                .execute()
    print("now i completed the merge")

else:
    print("i'll load all the data to the gold layer")
    df_model_final.write.format("delta").mode("overwrite").option("path", f"abfss://gold@ataacarprojectdatalake.dfs.core.windows.net/{dim_name}/").save()
    inc_flag = True

now i completed the merge


In [0]:
%sql
select * from delta.`abfss://gold@ataacarprojectdatalake.dfs.core.windows.net/dim_date/`

Date_ID,Date_sk
DT00029,1
DT00140,2
DT00192,3
DT00444,4
DT00475,5
DT00947,6
DT00976,7
DT01028,8
DT01099,9
DT00657,10


In [0]:
display(df_src)

Branch_ID,BranchName
BR0131,Audi Motors
BR0760,Healey Motors
BR0789,Hillman Motors
BR0938,Isotta Fraschini Motors
BR1040,Lada Motors
BR1693,Saleen Motors
BR1792,Simca do Brasil Motors
BR1799,Simca do Brasil Motors
BR1955,Toyota Motors
BR1978,Turner Motors
